## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-18-56-17 +0000,nyt,How Louisiana Became ICE Detention Central,https://www.nytimes.com/video/us/1000000102342...
1,2025-07-31-18-56-08 +0000,nypost,Judges skeptical of Trump using emergency powe...,https://nypost.com/2025/07/31/us-news/judges-s...
2,2025-07-31-18-53-04 +0000,wapo,Texas lawmakers fault Kerr County officials on...,https://www.washingtonpost.com/nation/2025/07/...
3,2025-07-31-18-52-59 +0000,nypost,Long Island Hooters has become one of the sexy...,https://nypost.com/2025/07/31/us-news/long-isl...
4,2025-07-31-18-52-17 +0000,startribune,"With season slipping away, Twins stock up for ...",https://www.startribune.com/minnesota-twins-ml...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2359/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,79
46,tariffs,24
92,trade,20
446,deal,17
522,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
163,2025-07-31-10-33-13 +0000,cbc,"U.S. extends trade deal with Mexico, sets tari...",https://www.cbc.ca/news/world/trump-tariffs-so...,182
279,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,164
75,2025-07-31-16-29-38 +0000,latimes,New Trump tariffs on Mexico delayed 90 days as...,https://www.latimes.com/world-nation/story/202...,159
269,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...,150
146,2025-07-31-12-28-38 +0000,wapo,Trump puts trade deal in doubt after Canada ba...,https://www.washingtonpost.com/politics/2025/0...,149


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
163,182,2025-07-31-10-33-13 +0000,cbc,"U.S. extends trade deal with Mexico, sets tari...",https://www.cbc.ca/news/world/trump-tariffs-so...
342,94,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
125,52,2025-07-31-14-01-19 +0000,nypost,Fed’s preferred inflation gauge ticks up as ta...,https://nypost.com/2025/07/31/business/feds-pr...
101,52,2025-07-31-15-31-00 +0000,wsj,President Trump’s tighter deadline for Russia ...,https://www.wsj.com/politics/policy/trumps-rus...
335,52,2025-07-30-20-33-00 +0000,wsj,"Ford Takes $800 Million Tariff Hit, Posts Firs...",https://www.wsj.com/business/autos/ford-earnin...
269,51,2025-07-30-23-08-32 +0000,cbc,Trump slams Canada's plan to recognize Palesti...,https://www.cbc.ca/news/politics/us-canada-tra...
283,48,2025-07-30-22-31-15 +0000,nypost,Witkoff could visit Gaza aid sites during Isra...,https://nypost.com/2025/07/30/world-news/witko...
95,47,2025-07-31-15-49-33 +0000,nypost,Left melts down over new Dunkin’ ‘genetics’ ad...,https://nypost.com/2025/07/31/us-news/left-mel...
256,40,2025-07-30-23-37-48 +0000,wapo,NTSB hearing: Army helicopter’s location syste...,https://www.washingtonpost.com/business/2025/0...
190,36,2025-07-31-08-34-03 +0000,nypost,"Russian missile and drone attack batters Kyiv,...",https://nypost.com/2025/07/31/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
